In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils, print_summary
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import pandas as pd
from sklearn.model_selection import train_test_split
import keras.backend as K

Using TensorFlow backend.


In [22]:
num_of_classes = 8

In [25]:
def keras_model(image_x, image_y):
    global num_of_classes
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(image_x, image_y, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(64, (5, 5), activation='sigmoid'))
    model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(num_of_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath = "emojinator.h5"
    checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint1]

    return model, callbacks_list


Copy paste the below cell in python file say "gen_images.py" and run it via command line : python gen_images.py

In [2]:
import cv2
import numpy as np
import os

image_x, image_y = 50, 50

cap = cv2.VideoCapture(0)
fbag = cv2.createBackgroundSubtractorMOG2()

def create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)

def main(g_id):
    total_pics = 1200
    cap = cv2.VideoCapture(0)
    x, y, w, h = 300, 50, 350, 350

    create_folder("gestures/" + str(g_id))
    pic_no = 0
    flag_start_capturing = False
    frames = 0

    while True:
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(frame, frame, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening=cv2.morphologyEx(dilation,cv2.MORPH_CLOSE,kernel_square)

        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)
        thresh = thresh[y:y + h, x:x + w]
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]

        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 10000 and frames > 50:
                x1, y1, w1, h1 = cv2.boundingRect(contour)
                pic_no += 1
                save_img = thresh[y1:y1 + h1, x1:x1 + w1]
                if w1 > h1:
                    save_img = cv2.copyMakeBorder(save_img, int((w1 - h1) / 2), int((w1 - h1) / 2), 0, 0,
                                                  cv2.BORDER_CONSTANT, (0, 0, 0))
                elif h1 > w1:
                    save_img = cv2.copyMakeBorder(save_img, 0, 0, int((h1 - w1) / 2), int((h1 - w1) / 2),
                                                  cv2.BORDER_CONSTANT, (0, 0, 0))
                save_img = cv2.resize(save_img, (image_x, image_y))
                cv2.putText(frame, "Capturing...", (30, 60), cv2.FONT_HERSHEY_TRIPLEX, 2, (127, 255, 255))
                cv2.imwrite("gestures/" + str(g_id) + "/" + str(pic_no) + ".jpg", save_img)

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, str(pic_no), (30, 400), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (127, 127, 255))
        cv2.imshow("Capturing gesture", frame)
        cv2.imshow("thresh", thresh)
        keypress = cv2.waitKey(1)
        if keypress == ord('c'):
            if flag_start_capturing == False:
                flag_start_capturing = True
            else:
                flag_start_capturing = False
                frames = 0
        if flag_start_capturing == True:
            frames += 1
        if pic_no == total_pics:
            break


g_id = input("Enter gesture number: ")
main(g_id)

Enter gesture number: 0


Below cell will convert the images into numpy array and store it in csv file

In [1]:
from scipy.misc import imread
import numpy as np
import pandas as pd
import os
root = './gestures' # or ‘./test’ depending on for which the CSV is being created

# go through each directory in the root folder given above
for directory, subdirectories, files in os.walk(root):
# go through each file in that directory
    for file in files:
    # read the image file and extract its pixels
        if int(file[:-4]) == 1200:
            print(file , directory[11:])
        im = imread(os.path.join(directory,file))
        value = im.flatten()
# I renamed the folders containing digits to the contained digit itself. For example, digit_0 folder was renamed to 0.
# so taking the 9th value of the folder gave the digit (i.e. "./train/8" ==> 9th value is 8), which was inserted into the first column of the dataset.
        value = np.hstack((directory[11:],value))
        df = pd.DataFrame(value).T
        df = df.sample(frac=1) # shuffle the dataset
        with open('train_foo.csv', 'a') as dataset:
            df.to_csv(dataset, header=False, index=False)

1.jpg 0
10.jpg 0
100.jpg 0
1000.jpg 0
1001.jpg 0
1002.jpg 0
1003.jpg 0
1004.jpg 0
1005.jpg 0
1006.jpg 0
1007.jpg 0
1008.jpg 0
1009.jpg 0
101.jpg 0
1010.jpg 0
1011.jpg 0
1012.jpg 0
1013.jpg 0
1014.jpg 0
1015.jpg 0
1016.jpg 0
1017.jpg 0
1018.jpg 0
1019.jpg 0
102.jpg 0
1020.jpg 0
1021.jpg 0
1022.jpg 0
1023.jpg 0
1024.jpg 0
1025.jpg 0
1026.jpg 0
1027.jpg 0
1028.jpg 0
1029.jpg 0
103.jpg 0
1030.jpg 0
1031.jpg 0
1032.jpg 0
1033.jpg 0
1034.jpg 0
1035.jpg 0
1036.jpg 0
1037.jpg 0
1038.jpg 0
1039.jpg 0
104.jpg 0
1040.jpg 0
1041.jpg 0
1042.jpg 0
1043.jpg 0
1044.jpg 0
1045.jpg 0
1046.jpg 0
1047.jpg 0
1048.jpg 0
1049.jpg 0
105.jpg 0
1050.jpg 0
1051.jpg 0
1052.jpg 0
1053.jpg 0
1054.jpg 0
1055.jpg 0
1056.jpg 0
1057.jpg 0
1058.jpg 0
1059.jpg 0
106.jpg 0
1060.jpg 0
1061.jpg 0
1062.jpg 0
1063.jpg 0
1064.jpg 0
1065.jpg 0
1066.jpg 0
1067.jpg 0
1068.jpg 0
1069.jpg 0
107.jpg 0
1070.jpg 0
1071.jpg 0
1072.jpg 0
1073.jpg 0
1074.jpg 0
1075.jpg 0
1076.jpg 0
1077.jpg 0
1078.jpg 0
1079.jpg 0
108.jpg 0
1080.jpg 0
10

This cell will take the stored images data from csv file and split it for traning and test data

In [15]:
dataset = pd.read_csv("train_foo.csv")
# Normalizing the training dataset (train.csv file)# Norma 
dataset = dataset.sample(frac=1)
X = dataset.iloc[:,1:]
Y = np.array(dataset.iloc[:,0])
X = X/255.                      # for normalizing the inputs
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=123, train_size = 0.8)

# Reshape

print("number of training examples = " + str(X_train.shape[0]))
print("number of test examples = " + str(X_test.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))

number of training examples = 7679
number of test examples = 1920
X_train shape: (7679, 2500)
Y_train shape: (7679,)
X_test shape: (1920, 2500)
Y_test shape: (1920,)


C:\Users\Shree\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


This cell will train the model and evaluate its performance

In [ ]:

def main():
    dataset = pd.read_csv("train_foo.csv")
    # Normalizing the training dataset (train.csv file)# Norma 
    dataset = dataset.sample(frac=1)
    X = dataset.iloc[:,1:]
    Y = np.array(dataset.iloc[:,0])
    X = X/255.                      # for normalizing the inputs
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=123, train_size = 0.8)
    
    #X_train = X_train.T
    #X_test = X_test.T
    print(Y_train.shape[0])
    Y_train = Y_train.reshape(Y_train.shape[0],1)
    Y_test = Y_test.reshape(-1,1)
    # Reshape
    print("number of training examples = " + str(X_train.shape[0]))
    print("number of test examples = " + str(X_test.shape[0]))
    print("X_train shape: " + str(X_train.shape))
    print("Y_train shape: " + str(Y_train.shape))
    print("X_test shape: " + str(X_test.shape))
    print("Y_test shape: " + str(Y_test.shape))
    image_x = 50
    image_y = 50

    import ipdb
    #ipdb.set_trace()
    train_y = np_utils.to_categorical(Y_train)
    test_y = np_utils.to_categorical(Y_test)
    train_y = train_y.reshape(train_y.shape[0], train_y.shape[1])
    test_y = test_y.reshape(test_y.shape[0], test_y.shape[1])
    X_train = np.array(X_train)
    X_train = X_train.reshape(X_train.shape[0], 50, 50, 1)
    X_test = np.array(X_test)
    X_test = X_test.reshape(X_test.shape[0], 50, 50, 1)
    print("X_train shape: " + str(X_train.shape))
    print("X_test shape: " + str(X_test.shape))

    model, callbacks_list = keras_model(image_x, image_y)
    model.fit(X_train, train_y, validation_data=(X_test, test_y), epochs=10, batch_size=64,
              callbacks=callbacks_list)
    scores = model.evaluate(X_test, test_y, verbose=0)
    print("CNN Error: %.2f%%" % (100 - scores[1] * 100))
    print_summary(model)

    model.save('emojinator.h5')

main()

C:\Users\Shree\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


7679
number of training examples = 7679
number of test examples = 1920
X_train shape: (7679, 2500)
Y_train shape: (7679, 1)
X_test shape: (1920, 2500)
Y_test shape: (1920, 1)
X_train shape: (7679, 50, 50, 1)
X_test shape: (1920, 50, 50, 1)
Train on 7679 samples, validate on 1920 samples
Epoch 1/10
7679/7679 [==============================] - 88s 11ms/step - loss: 1.0072 - acc: 0.6763 - val_loss: 0.1295 - val_acc: 0.9589

In [1]:
import cv2
from keras.models import load_model
import numpy as np
import os

model = load_model('emojinator.h5')

def main():
    emojis = get_emojis()
    cap = cv2.VideoCapture(0)
    x, y, w, h = 300, 50, 350, 350

    while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(img, img, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

        thresh = thresh[y:y + h, x:x + w]
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 2500:
                x, y, w1, h1 = cv2.boundingRect(contour)
                newImage = thresh[y:y + h1, x:x + w1]
                newImage = cv2.resize(newImage, (50, 50))
                pred_probab, pred_class = keras_predict(model, newImage)
                print(pred_class, pred_probab)
                img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

        x, y, w, h = 300, 50, 350, 350
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            break

def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class


def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

def get_emojis():
    emojis_folder = 'hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

keras_predict(model, np.zeros((50, 50, 1), dtype=np.uint8))
main()



Using TensorFlow backend.


0
1
2
3
4
5
6
7
8
0 0.999999
0 1.0
0 1.0
0 0.99996
0 0.621901
3 0.940016
0 0.909237
0 0.917157
0 0.937877
0 0.989789
0 0.999818
0 0.999322
6 0.530243
0 0.987073
6 0.927617
0 0.998429
0 0.999899
0 1.0
0 0.999996
0 0.999992
0 0.999982
0 0.999999
0 0.999998
0 1.0
0 0.999954
0 0.999809
0 0.999907
0 0.999934
0 0.999988
0 0.999692
0 1.0
0 1.0
0 1.0
0 1.0
0 1.0
0 0.878523
0 0.821402
0 0.988165
0 0.999818
0 0.99931
0 0.999605
0 0.999519
0 0.999551
0 0.99963
0 0.999508
0 0.999917
0 0.999296
0 0.999413
0 0.999868
0 0.99995
0 0.999758
0 0.999831
0 0.999995
0 0.999936
0 0.999983
0 0.99999
0 0.999993
0 0.999994
0 0.999994
0 0.999993
0 0.999979
0 0.999988
0 0.999972
0 0.999995
0 0.999992
0 0.999985
0 0.999988
0 0.999989
0 0.999865
0 0.999992
0 0.999981
0 0.999931
0 0.999964
0 0.999888
0 0.999923
0 0.99999
0 0.999895
0 0.999502
0 0.999964
0 0.999898
0 0.999988
0 0.999651
0 0.999956
0 0.999937
0 0.999627
0 0.999831
0 0.999619
0 0.999171
0 0.999778
0 0.999941
0 0.999537
0 0.999893
0 1.0
0 1.0
0 1.0
0 1

KeyboardInterrupt: 